# Using spatial indexing for faster querying of intersection points

In [18]:
import re
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
from glob import glob
from shapely.geometry import Point, Polygon
from xml.etree import ElementTree as ET

Let us create first a grid of points covering the earth

In [24]:
latitudeArray = np.linspace(-90.0, 90.0, 721)
longitudeArray = np.linspace(0.0, 360.0, 1441)
gridCoordinates = {'x':[], 'y':[]}
for i in longitudeArray:
    for j in latitudeArray:
        gridCoordinates['x'].append(i)
        gridCoordinates['y'].append(j)
with open('./Dataset/pickles/grid_coordinates.p', 'wb') as dest:
    pickle.dump(gridCoordinates, dest)

Let's convert the `gridCoordinates` dictionary into pandas dataframe and then into geopandas dataframe

In [31]:
df = pd.DataFrame.from_dict(gridCoordinates, orient='columns')
df['geometry'] = df.apply(lambda dframe: Point(dframe['x'], dframe['y']), axis=1)
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=df['geometry'])
gdf.head()

,x,y,geometry
0,0.0,-90.00,POINT (0 -90)
1,0.0,-89.75,POINT (0 -89.75)
2,0.0,-89.50,POINT (0 -89.5)
3,0.0,-89.25,POINT (0 -89.25)
4,0.0,-89.00,POINT (0 -89)


Let's calculate the spatial index for the geodataframe

In [39]:
gridSpatialIndex = gdf.sindex
with open('./Dataset/pickles/grid_spatial_index.p', 'wb') as dest:
    pickle.dump(gridSpatialIndex, dest)

Now let's read in the kml file that will be used as a mask

In [40]:
kmlFpath = glob('./Dataset/kml/*.kml')[0]
tree = ET.parse(kmlFpath)
root = tree.getroot().tag
kmlns = re.split('[{}]', root)[1]
elements = tree.findall(".//{%s}coordinates" % kmlns)
for el in elements:
    shapeCoordinates = []
    for coords in el.text.split(' '):
        x, y = coords.split(',')
        x, y = float(x), float(y)
        x = (360+x) if x<0 else x
        shapeCoordinates.append((x,y))
maskPolygon = Polygon(shapeCoordinates)
print(maskPolygon.bounds)

(270.8512061, 40.0488429, 271.4252483, 40.2831503)


Here, we carry out the querrying process in two passes. 
In the first pass, we carryout an approximate querying to filter those grid points that lie within the bounds of the polygon. In the second pass, we take these filtered points and then carry out accurate querying on them to see if they lie inside the polygon or not. PostGIS carries out similar process. This part of the code has been inspired from [here](https://geoffboeing.com/2016/10/r-tree-spatial-index-python/).

In [46]:
possible_matches_index = list(gridSpatialIndex.intersection(maskPolygon.bounds))
possible_matches = gdf.iloc[possible_matches_index]
precise_matches = possible_matches[possible_matches.intersects(maskPolygon)]
print(precise_matches)

             x      y              geometry
782085  271.00  40.25     POINT (271 40.25)
782806  271.25  40.25  POINT (271.25 40.25)


Ain't this beautiful? 
If you want to learn more about spatial indexing, click [here](https://blog.mapbox.com/a-dive-into-spatial-search-algorithms-ebd0c5e39d2a). 